# Pyscript Jupyter Tutorial

## Contents
These links should work in a live Jupyter notebook, but not the Github viewer:
* [The Basics](#The-Basics)
* [State Variables](#State-Variables)
* [Triggers](#Triggers)
    * [State Trigger](#State-Trigger)
    * [Service Trigger](#Service-Trigger)
    * [Time Trigger](#Time-Trigger)
    * [Call function on startup and reload](#Call-function-on-startup-and-reload)
    * [Event Trigger](#Event-Trigger)
* [Task Functions](#Task-Functions)
    * [Task Unique](#Task-Unique)
    * [Task Wait Until](#Task-Wait-Until)
* [Global Contexts](#Global-Contexts)

## The Basics
When Juptyer notebook is connected to pyscript in HASS, it's similar to  being connected to a regular iPython interpreter.  All of pyscript's Python language features are available interactively.

Assuming you have downloaded this notebook and you have [installed the pyscript Jupyter kernel](https://github.com/custom-components/pyscript/tree/master/jupyter), you can run it interactively:
```bash
jupyter notebook pyscript_tutorial.ipynb
```

It's assumed you know the basics of Jupyter.  To execute a cell, hit `<Shift>Enter`.  You might prefer to clear all the cell results (Cell -> All Output -> Clear) before going through the tutorial.

First, we can do some simple math to make sure we are connected to the kernel:

In [1]:
1 + 2

3

To display messages use the `log.debug` (the `print` function is not currently supported)

In [2]:
log.debug("this text will be displayed")

this text will be displayed


Often you'll want to include variables in the displayed message, which can be done as follows:

In [3]:
x = 10
y = "a string"
list1 = [1, 2, 3]
log.debug(f"this is a formatted string with variables x = {x}, y = {y}, list1 = {list1}")

this is a formatted string with variables x = 10, y = a string, list1 = [1, 2, 3]


There are also related functions `log.info`, `log.warn` and `log.error` to display messages of increasing importance.  When you are running a Jupyter session, all of these log functions display their output interactively.  The messages will also appear in the HASS log file, based on the log level configuation.

## State Variables
HASS state variables can be accessed by name.  For example, the current state of the sun is:

In [4]:
sun.sun

'above_horizon'

In the HASS UI, go to Developer Tools and select that States tab to see the available state variables. State variables have attributes that can be accessed by appending the attribute name.  For example, the sun's evevations is:

In [5]:
sun.sun.elevation

62.6

Auto-completion of variable and function names is supported.  After typing the first part of the name, hit `TAB` to see the potential completions. That's a convenient way to see state variables and their attributes. Before running this next example, put your cursor at the end of the line, hit `TAB`, and select one of the options before running:

In [ ]:
sun.sun.

You can set a state variable by assigning to it (watch the States tab in the UI to see the new state variable):

In [6]:
pyscript.example_var = 20

Note that all HASS state variables have names forced to lower-case and values forced to strings.  The value of the variable we just created is `'20'` (a str), not `20` (an int):

In [7]:
pyscript.example_var

'20'

This means that statements like this will fail with an error:

In [8]:
if pyscript.example_var > 10:
    log.debug("pyscript.example_var is greater than 10")

TypeError: '>' not supported between instances of 'str' and 'int'

So instead you need to cast the value to an integer, assuming you are sure the string really does contain a valid integer:

In [9]:
if int(pyscript.example_var) > 10:
    log.debug(f"pyscript.example_var = {pyscript.example_var}, which is greater than 10")

pyscript.example_var = 20, which is greater than 10


The functions `state.set` and `state.get` can be used to set and get state variable values, in case you want
to compute the name dynamically.  The `state.set` function also allows you to set attributes by including them
in a `dict`, and they maintain their Python types:

In [10]:
state.set("pyscript.example_var", 30, attr1=30, attr2=23.7, attr3="abc")

You can see the new attributes in the States tab, or just check them here:

In [11]:
pyscript.example_var.attr2

23.7

You can define functions, use loops and `if` statements.  Let's define a function `func1` which returns two different calculations based on the sun's elevation (there's no need to cast the attribute value, since it's already `float`):

In [12]:
def func1(x, y):
    if sun.sun.elevation > 30:
        log.info(f"The sun's elevation is {sun.sun.elevation}, which is > 30 degrees")
        return x + 2 * y
    else:
        log.info(f"The sun's elevation is {sun.sun.elevation}, which is <= 30 degrees")
        return x - y

This function should be called with two arguments:

In [13]:
func1(1, 2)

5

The sun's elevation is 62.6, which is > 30 degrees


## Triggers

### State Trigger
The `@state_trigger` decorator specifies a Python expression that is evaluated each time the state variables it mentions change, and it calls the function when it evaluates to `True`.  Like all trigger functions, the function runs as a new asyncronous task. Here's an example:

In [14]:
@state_trigger("pyscript.state_trig_run == '1'")
def state_trig_func(**kwargs):
    log.info(f"state_trig_func called with kwargs={kwargs}")

This function will be run whenever the state variable `pyscript.state_trig_run` is set to `'1'` or '`'.  We can test it by manually setting the variable, first to a value that doesn't cause the trigger:

In [15]:
pyscript.state_trig_run = 2

and next to the value that does cause it to run:

In [16]:
pyscript.state_trig_run = 1

state_trig_func called with kwargs={'trigger_type': 'state', 'var_name': 'pyscript.state_trig_run', 'value': '1', 'old_value': '2'}


Notice that function is called with several keyword parameters that capture the type of trigger, the name of the state variable that caused the trigger (useful if the expression includes several state variables) and its old and new values.

Notice if you repeat that last assignment, the function doesn't get called again. HASS doesn't notify a state variable change if the value is unchanged. So to make the trigger happen again, you need to run the prior cell that sets `pyscript.state_trig_run` to a different value

You can optionally specify `@state_active` and `@time_active` decorators.  These are checked whenever the trigger condition occurs (whether state, time or event), and the function will only be called if the active conditions are met.  For example, we can add these conditions to the previous example:

In [17]:
@state_trigger("pyscript.state_trig_run == '1'")
@state_active("pyscript.state_trig_active == '10'")
@time_active("range(3:00, 23:00)")
def state_trig_func(**kwargs):
    log.info(f"state_trig_func called with kwargs={kwargs}")

The trigger will only happen if `pyscript.state_trig_active` is `'10'` and it's between 3am and 11pm.  First let's make sure it doesn't run with `pyscript.state_trig_active` set to the wrong value:

In [18]:
pyscript.state_trig_active = 5
pyscript.state_trig_run = 2
pyscript.state_trig_run = 1

Now we'll set it to the correct value and make the trigger occur again (assuming it's between 3am and 11pm; if not adjust the time range above):

In [19]:
pyscript.state_trig_active = 10
pyscript.state_trig_run = 2
pyscript.state_trig_run = 1

state_trig_func called with kwargs={'trigger_type': 'state', 'var_name': 'pyscript.state_trig_run', 'value': '1', 'old_value': '2'}


Finally, let's clean up at the end of this section by deleting the function:

In [20]:
del state_trig_func

### Service Trigger
The `@service` decorator causes the function to be registered as a HASS service.  The service name has a `DOMAIN` of `pyscript` and the service is the function name, so the full service name is `pyscript.<func_name>`. Whenever that service is called, the function is called with any parameters from the service call.

In [21]:
@service
def test_service(param1=None, other_param=None):
    log.debug(f"test_service called with param1={param1}, other_param={other_param}")

The service can be called using the full service name as a function. You can also call the service from the Services tab in the Developers page of the HASS UI. These YAML arguments will specify the same parameters when calling the service
from the HASS UI:
```yaml
param1:
  45
other_param:
  - 6
  - 8
  - 12
```

In [22]:
pyscript.test_service(param1=45, other_param=[6,8,12])

test_service called with param1=45, other_param=[6, 8, 12]


Alternatively, the `service.call` function can be used to call the service using the separate `DOMAIN` and service name:

In [24]:
service.call("pyscript", "test_service", param1=45, other_param=[6,8,12])

test_service called with param1=45, other_param=[6, 8, 12]


The service can be removed by undefining the Python function, or redefining it without the `@service` decorator:

In [25]:
del test_service

You should now see that the service is no longer listed in the HASS UI, and you can check using `service.has_service`:

In [26]:
service.has_service("pyscript", "test_service")

False

If you are not sure what arguments are passed into a service or trigger function, define it with the `**kwargs` argument, which captures all of the keywords arguments into a `dict`.  For example, we can redefine the service function above, and when it is called you will see the `dict` displayed.  One small drawback with doing this is that the service call parameters are not know by HASS, so the UI won't prompt for them.

In [27]:
@service
def test_service2(**kwargs):
    log.debug(f"test_service called with parameters {kwargs}")

service.call("pyscript", "test_service2", param1=45, other_param=[6,8,12])

test_service called with parameters {'param1': 45, 'other_param': [6, 8, 12]}


### Time Trigger

This function triggers every 5 seconds (starting at midnight every day, which is the same as continuously), but only calls the function if the condition `pyscript.ok1 == '1'` is met.

In [28]:
@time_trigger("period(0:00, 5 sec)")
@state_active("pyscript.ok1 == '1'")
def every_5_seconds():
    log.debug("every_5_seconds: running now")

The trigger is now occuring every 5 seconds, but the function isn't run on each trigger since the `@state_active` condition isn't `True`.  Let's set that state variable to 1, and now wait a short while to look for the output:

In [29]:
pyscript.ok1 = 1

every_5_seconds: running now
every_5_seconds: running now
every_5_seconds: running now


Now turn off the `@state_active` condition, and after waiting a while you can confirm the function isn't being run:

In [30]:
pyscript.ok1 = 0

To disable the function, either delete it or redefine it without any triggers:

In [31]:
del every_5_seconds

### Call function on startup and reload

With a bare `@time_trigger` the function is called on startup or load.  In Jupyter that happens whenever you
define the function by running the cell it is defined in.  When you run this cell, the function is called
immediately.

In [32]:
@time_trigger
def on_startup():
    log.debug("on_startup: running")
    task.sleep(2)
    log.debug("on_startup: finished")

on_startup: running
on_startup: finished


### Event Trigger
Event triggers are defined with the `@event_trigger` decorator. The first argument is the event to trigger on, and the optional second argument is a Python expression based on state variables and event parameters that is evaluated when each trigger occurs, and the function is called if the condition is `True`.  Here's an example:

In [33]:
@event_trigger("my_event")
def got_my_event(**kwargs):
    log.info(f"got my_event: kwargs={kwargs}")

You can test this by firing an event with the `event.fire` function, or using the Events tab in the Developer page of the HASS UI.

In [34]:
event.fire("my_event", my_param1=100, fruit="apple")

got my_event: kwargs={'trigger_type': 'event', 'event_type': 'my_event', 'my_param1': 100, 'fruit': 'apple'}


You can see the function is called with the two event parameters, and also a handful of other parameters that
tell the function the type of trigger and name of the event.

Rather than use the catch-all `**kwargs`, you could explicitly specify the parameters.  In this example we'll also add a trigger condition, so that the function only runs when `my_param1` is between 50 and 150 (you could also include state variables in the condition if you want):

In [35]:
@event_trigger("my_event", "50 <= my_param1 <= 150")
def got_my_event(my_param1=None, fruit=None):
    log.info(f"got my_event: my_param1={my_param1}, fruit={fruit}")

Now fire the event three times; you should see that only one of the events causes the function to run:

In [36]:
event.fire("my_event", my_param1=40, fruit="apple")
event.fire("my_event", my_param1=110, fruit="orange")
event.fire("my_event", my_param1=180, fruit="banana")

got my_event: my_param1=110, fruit=orange


As discussed in this [Wiki Page](https://github.com/custom-components/pyscript/wiki/Event-based-triggers), here's an example that listens to state change events for a particular component, in this case `sun`.  We trigger on the built-in `EVENT_STATE_CHANGED` event, and only call the function when the state variable name starts with `"sun."`.  There are typically a pair state updates every 4-5 minutes, so you might need to wait a while to see any output:

In [37]:
from homeassistant.const import EVENT_STATE_CHANGED

@event_trigger(EVENT_STATE_CHANGED, "entity_id.startswith('sun.')") 
def monitor_state_change(entity_id=None, new_state=None, old_state=None):
    old_value = old_state.state if old_state else None
    log.info(f"entity {entity_id} changed from {old_value} to {new_state.state} and attributes are now {new_state.attributes}")

entity sun.sun changed from above_horizon to above_horizon and attributes are now {'next_dawn': '2020-08-24T13:05:35+00:00', 'next_dusk': '2020-08-24T03:18:39+00:00', 'next_midnight': '2020-08-24T08:11:10+00:00', 'next_noon': '2020-08-24T20:11:24+00:00', 'next_rising': '2020-08-24T13:32:46+00:00', 'next_setting': '2020-08-24T02:51:24+00:00', 'elevation': 62.38, 'azimuth': 196.92, 'rising': False, 'friendly_name': 'Sun'}
entity sun.sun changed from above_horizon to above_horizon and attributes are now {'next_dawn': '2020-08-24T13:05:35+00:00', 'next_dusk': '2020-08-24T03:18:39+00:00', 'next_midnight': '2020-08-24T08:11:10+00:00', 'next_noon': '2020-08-24T20:11:24+00:00', 'next_rising': '2020-08-24T13:32:46+00:00', 'next_setting': '2020-08-24T02:51:24+00:00', 'elevation': 62.38, 'azimuth': 196.93, 'rising': False, 'friendly_name': 'Sun'}


If you are tired of waiting, you could set some random state variable that starts with `"sun."`, although it's not recommended to set state variable in a components address space:

In [38]:
sun.my_new_var = 13

entity sun.my_new_var changed from None to 13 and attributes are now {}


It's much better to use `@state_trigger` decorators to trigger on state variable changes, rather than triggering on all `EVENT_STATE_CHANGED` events as we did in this example.

Finally, let's delete this event trigger before moving on:

In [39]:
del monitor_state_change

## Task Functions

### Task Unique
There are seveal features that allow functions to sleep, wait for additional triggers or kill other tasks. Pyscript does all functions that are triggered to be long-running since they run as separate asynhronous tasks.  Let's create a service to demonstrate some of these features:

In [40]:
@service
def long_running(name="task", loop_cnt=10):
    log.debug(f"{name}: starting")
    for i in range(loop_cnt):
        task.sleep(3)
        log.debug(f"{name}:     completed loop #{i}")
    log.debug(f"{name}: finished")

This function runs a certain number of loops, each of which sleeps for 3 seconds and displays a message.  Let's test it by calling the service:

In [41]:
pyscript.long_running(loop_cnt=3)

task: starting
task:     completed loop #0
task:     completed loop #1
task:     completed loop #2
task: finished


The service triggers the function, which runs as a new asynhronous task.  Let's do two service calls, using the optional `name` parameter so we can distinguish the output:

In [42]:
pyscript.long_running(name="task0", loop_cnt=3)
pyscript.long_running(name="task1", loop_cnt=4)

task0: starting
task1: starting
task0:     completed loop #0
task1:     completed loop #0
task0:     completed loop #1
task1:     completed loop #1
task0:     completed loop #2
task0: finished
task1:     completed loop #2
task1:     completed loop #3
task1: finished


You can see that each function is running independently, starting at almost the same time.  The second service call runs one more loop so it finishes about 3 seconds after the first.

Depending on our application, we might prefer that there is only one such long-running function.  We can accomplish this with `task.unique`. When `task.unique` is called with a string argument, any task that is currently running that called `task.unique` with that same agument will be killed. Let's add a call to `task.unique` to our function:

In [43]:
@service
def long_running(name="task", loop_cnt=10):
    task.unique("long_running_service")
    log.debug(f"{name}: starting")
    for i in range(loop_cnt):
        task.sleep(3)
        log.debug(f"{name}:     completed loop #{i}")
    log.debug(f"{name}: finished")

Now let's start a longer service run:

In [44]:
pyscript.long_running(loop_cnt=10)

task: starting
task:     completed loop #0
task:     completed loop #1
task:     completed loop #2


and after you've seen a couple of lines of output, let's call `task.unique` with the same argument:

In [45]:
task.unique("long_running_service")

Normally you wouldn't call `task.unique` outside a function, but it is a good way to stop an existing long-running function, since redefining it won't stop the current running instance.  Now let's make two service calls like before, and see what happens. We wait a short while before the second sevice call:

In [46]:
pyscript.long_running(name="task0", loop_cnt=3)
task.sleep(0.5)
pyscript.long_running(name="task1", loop_cnt=4)

task0: starting
task1: starting
task1:     completed loop #0
task1:     completed loop #1
task1:     completed loop #2
task1:     completed loop #3
task1: finished


Notice that `task0` does start, but once `task1` starts, its call to `task.unique` causes `task0` to be killed.

`task.unique` takes an argument that reverses the behavior - the new task is killed and the old one keeps running:

In [47]:
@service
def long_running(name="task", loop_cnt=10):
    task.unique("long_running_service", kill_me=True)
    log.debug(f"{name}: starting")
    for i in range(loop_cnt):
        task.sleep(3)
        log.debug(f"{name}:     completed loop #{i}")
    log.debug(f"{name}: finished")

Now we'll call the same service twice:

In [48]:
pyscript.long_running(name="task0", loop_cnt=3)
task.sleep(0.5)
pyscript.long_running(name="task1", loop_cnt=4)

task0: starting
task0:     completed loop #0
task0:     completed loop #1
task0:     completed loop #2
task0: finished


As expected, the first service call continues and the second service call is terminated as soon as it calls `task.unique`.

### Task Wait Until
The function `task.wait_until` can be used to wait for additional state, time or event triggers. Three keyword arguments, `state_trigger`, `time_trigger` and `event_trigger` match the corresponding trigger decorators. There is an overall optional `timeout` in seconds, which causes `task.wait_until` to return when the timeout expires if no triggers have occurred. See the [README](https://github.com/custom-components/pyscript#waiting-for-events) for more information.

In this example, we have two binary sensors and we want to know if sensor2 is active within 10 seconds of sensor1 becoming active:

In [49]:
@state_trigger("input_boolean.test_sensor1 == 'on'")
def sensor1_active():
    task.unique('sensor1_active')
    log.debug("waiting for sensor2")
    result = task.wait_until(state_trigger="input_boolean.test_sensor2 == 'on'", timeout=10)
    log.debug(f"result = {result}")

Let's turn both sensors off, turn on sensor 1, wait for less than 10 seconds, then turn sensor2 on:

In [50]:
# reset both sensors
input_boolean.test_sensor1 = "off"
input_boolean.test_sensor2 = "off"
# this will trigger the function
input_boolean.test_sensor1 = "on"
# wait a few seconds, then turn sensor2 on:
task.sleep(3)
input_boolean.test_sensor2 = "on"

waiting for sensor2
result = {'trigger_type': 'state', 'var_name': 'input_boolean.test_sensor2', 'value': 'on', 'old_value': 'off'}


You can see that `task.wait_until` returns as soon as sensor2 is set to `on`. The return `dict` shows what caused the function to return (in this case a state trigger), the name of the variable that caused the trigger, and its new and old values. If we re-run this cell with a 12 second delay, instead the timeout will cause the return:

In [51]:
# reset both sensors
input_boolean.test_sensor1 = "off"
input_boolean.test_sensor2 = "off"
# this will trigger the function
input_boolean.test_sensor1 = "on"
# wait a few seconds, then turn sensor2 on:
task.sleep(12)
input_boolean.test_sensor2 = "on"

waiting for sensor2
result = {'trigger_type': 'time'}


Now the result shows that the timeout caused `task.wait_until` to return.

Now assume we also want to make sure that sensor1 stays on the whole time we are waiting. We now check the return values and display the three different cases:

In [52]:
@state_trigger("input_boolean.test_sensor1 == 'on'")
def sensor1_active():
    task.unique('sensor1_active')
    log.debug("waiting for sensor2")
    result = task.wait_until(
        state_trigger="input_boolean.test_sensor2 == 'on' or input_boolean.test_sensor1 == 'off'",
        timeout=10
    )
    if result["trigger_type"] == "time":
        log.debug(f"sensor2 didn't turn on withing 10 seconds")
    elif result["trigger_type"] == "state":
        if result["var_name"] == "input_boolean.test_sensor2":
            log.debug(f"sensor2 turned on within 10 seconds of sensor1 turning on")
        else:
            log.debug(f"sensor1 turned off before sensor2 turned on")

Now let's test it out. In the first, normal, case, sensor2 turns 3 seconds after sensor1:

In [53]:
# reset both sensors
input_boolean.test_sensor1 = "off"
input_boolean.test_sensor2 = "off"
# this will trigger the function
input_boolean.test_sensor1 = "on"
# wait a few seconds, then turn sensor2 on:
task.sleep(3)
input_boolean.test_sensor2 = "on"

waiting for sensor2
sensor2 turned on within 10 seconds of sensor1 turning on


Now let's turn sensor1 back off before sensor2: 

In [54]:
# reset both sensors
input_boolean.test_sensor1 = "off"
input_boolean.test_sensor2 = "off"
# this will trigger the function
input_boolean.test_sensor1 = "on"
# wait a few seconds, then turn sensor1 off immediately before sensor2:
task.sleep(3)
input_boolean.test_sensor1 = "off"
input_boolean.test_sensor2 = "on"

waiting for sensor2
sensor1 turned off before sensor2 turned on


Finally, let's confirm the timeout case works too:

In [55]:
# reset both sensors
input_boolean.test_sensor1 = "off"
input_boolean.test_sensor2 = "off"
# this will trigger the function
input_boolean.test_sensor1 = "on"
# wait for a longer time, then turn sensor2 on:
task.sleep(12)
input_boolean.test_sensor2 = "on"

waiting for sensor2
sensor2 didn't turn on withing 10 seconds


## Global Contexts

You can list all the global contexts.  There is one for each pyscript script file, and one for each Jupyter session.  Our session is listed first:

In [56]:
pyscript.list_global_ctx()

['jupyter_0', 'file.scripts']

You can get the current global context:

In [57]:
pyscript.get_global_ctx()

'jupyter_0'

and set it: 

In [58]:
pyscript.set_global_ctx("file.scripts")

Let's set a global variable in this context, and switch back to our original one:

In [59]:
x_in_file_scripts = 123

In [60]:
x_in_file_scripts

123

Back in our original global context, we can confirm the variable doesn't exist:

In [61]:
pyscript.set_global_ctx("jupyter_0")
x_in_file_scripts

NameError: name 'x_in_file_scripts' is not defined